### 파이썬 미니 프로젝트 11조

In [2]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

#### 1. 데이터 수집

In [ ]:
load_dotenv()

True

In [10]:
# url과 파라미터 설정 + 결과 확인
url = 'http://apis.data.go.kr/1352000/ODMS_STAT_09/callStat09Api'
key = os.getenv("SERVICE_KEY")
params ={'serviceKey' : f'{key}', 'apiType' : 'JSON'}

response = requests.get(url, params=params)
print(response.content)

# 서버의 응답 상태 출력
print(response.status_code) 

b'{"pageNo":"1","resultCode":"00","totalCount":68,"items":[{"dsStdrSrvrt":"13.6","allSvr":"14","allStdrSrvrt":"9.3","year":"2016","dsSvr":"11","othdsStdrSrvrt":"6","dsSrvrt":"13.9","othdsSvr":"3","dvsd":"\xec\x84\xb8\xec\xa2\x85","allSrvrt":"11.9","othdsSrvrt":"7.9"},{"dsStdrSrvrt":"8.6","allSvr":"71","allStdrSrvrt":"5.5","year":"2016","dsSvr":"59","othdsStdrSrvrt":"2.8","dsSrvrt":"5.5","othdsSvr":"12","dvsd":"\xec\xa0\x84\xeb\x82\xa8","allSrvrt":"4.7","othdsSrvrt":"2.8"},{"dsStdrSrvrt":"10.8","allSvr":"37","allStdrSrvrt":"7.8","year":"2017","dsSvr":"30","othdsStdrSrvrt":"5.1","dsSrvrt":"6","othdsSvr":"7","dvsd":"\xec\xa0\x9c\xec\xa3\xbc","allSrvrt":"5.8","othdsSrvrt":"5.5"},{"dsStdrSrvrt":"11.1","allSvr":"92","allStdrSrvrt":"8.1","year":"2016","dsSvr":"86","othdsStdrSrvrt":"2.3","dsSrvrt":"8.5","othdsSvr":"6","dvsd":"\xeb\x8c\x80\xea\xb5\xac","allSrvrt":"7","othdsSrvrt":"2.1"},{"dsStdrSrvrt":"8.8","allSvr":"69","allStdrSrvrt":"6.2","year":"2016","dsSvr":"59","othdsStdrSrvrt":"2.4","ds

In [11]:
def api_df(num):
    
    url = f'http://apis.data.go.kr/1352000/ODMS_STAT_{num}/callStat{num}Api'
    params ={'serviceKey' : f'{key}', 
             'apiType' : 'JSON'}

    response = requests.get(url, params=params)
    # 응답 상태가 정상인 경우 (200)에 데이터프레임 반환
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['items'])
        return df
    else:
        print(f"오류 발생: {response.status_code}")

In [12]:
# 급성심장정지 발생률
heart_occur = api_df('08')

# 급성심장정지 생존율
heart_live = api_df('09')

# 급성심장정지 뇌기능 회복률
heart_heal = api_df('10')

In [13]:
# 전국 병원 데이터 csv 파일 불러오기
hospital_df = pd.read_csv("보건복지부_병원 및 의원 수_의료기관 종류별_시도별_20221231.csv", encoding='cp949')

#### 2. 데이터 전처리

In [16]:
# 급성심장정지 데이터프레임의 연도, 지역 컬럼을 앞으로 불러오기
front_cols = ['year', 'dvsd']
heart_occur = heart_occur[front_cols + [col for col in heart_occur.columns if col not in front_cols]]

heart_occur = heart_occur.rename(columns={
    'year': '연도',
    'dvsd': '시도',
    'allStdrOcrt': '전체 표준화 발생률',
    'othdsOcrt': '질병외 발생률',
    'dsStdrOcrt': '질병 표준화 발생률',
    'allOcrt': '전체 발생',
    'allOccr': '전체 발생률',
    'dsOcrt': '질병 발생률',
    'dsOccr': '질병 발생',
    'othdsStdrOcrt': '질병외 표준화 발생률',
    'othdsOccr': '질병외 발생'
})

heart_occur.head()

,연도,시도,전체 표준화 발생률,질병외 발생률,질병 표준화 발생률,전체 발생,전체 발생률,질병 발생률,질병 발생,질병외 표준화 발생률,질병외 발생
0,2018,강원,55.1,24.1,33.1,98,1503,66.8,1024,18.3,370
1,2018,충남,50.5,24.9,30.8,81.5,1718,55.9,1178,19.3,525
2,2018,경남,43,17.4,28.5,66.4,2229,47.8,1604,13.8,585
3,2016,부산,38.5,11.9,27.5,56.2,1947,42.5,1475,9.8,414
4,2016,제주,66.5,25.5,43.2,100.6,632,73.7,463,22.2,160


In [19]:
heart_live = heart_live[front_cols + [col for col in heart_live.columns if col not in front_cols]]

heart_live = heart_live.rename(columns={
    'year': '연도',
    'dvsd': '시도',
    'dsStdrSrvrt': '질병 표준화 생존률',
    'allSvr': '전체 생존',
    'allStdrSrvrt': '전체 표준화 생존률',
    'dsSvr': '질병 생존',
    'othdsStdrSrvrt': '질병외 표준화 생존률',
    'dsSrvrt': '질병 생존율',
    'othdsSvr': '질병외 생존',
    'allSrvrt': '전체 생존율',
    'othdsSrvrt': '질병외 생존율'
})

heart_live.head()

,연도,시도,질병 표준화 생존률,전체 생존,전체 표준화 생존률,질병 생존,질병외 표준화 생존률,질병 생존율,질병외 생존,전체 생존율,질병외 생존율
0,2016,세종,13.6,14,9.3,11,6,13.9,3,11.9,7.9
1,2016,전남,8.6,71,5.5,59,2.8,5.5,12,4.7,2.8
2,2017,제주,10.8,37,7.8,30,5.1,6,7,5.8,5.5
3,2016,대구,11.1,92,8.1,86,2.3,8.5,6,7,2.1
4,2016,전북,8.8,69,6.2,59,2.4,6.1,10,5.1,2.7


In [20]:
heart_heal = heart_heal[front_cols + [col for col in heart_heal.columns if col not in front_cols]]

heart_heal = heart_heal.rename(columns={
    'year': '연도',
    'dvsd': '시도',
    'allStbfnRcvr': '전체 표준화 뇌기능 회복률',
    'othdsStbfnRcvr': '질병외 표준화 뇌기능 회복률',
    'dsBrfr': '질병 뇌기능 회복',
    'allBrfr': '전체 뇌기능 회복',
    'dsBrfrr': '질병 뇌기능 회복률',
    'allBrfcRcvr': '전체 뇌기능 회복률',
    'dsStbfnRcvr': '질병 표준화 뇌기능 회복률',
    'othdsBrfr': '질병외 뇌기능 회복',
    'othdsBrfcRcvr': '질병외 뇌기능 회복률'
})


heart_heal.head()

,연도,시도,전체 표준화 뇌기능 회복률,질병외 표준화 뇌기능 회복률,질병 뇌기능 회복,전체 뇌기능 회복,질병 뇌기능 회복률,전체 뇌기능 회복률,질병 표준화 뇌기능 회복률,질병외 뇌기능 회복,질병외 뇌기능 회복률
0,2017,인천,6.8,1.6,77,82,6.8,5.6,10,5,1.6
1,2017,경북,3.6,1.7,48,58,3.1,2.7,5.4,9,1.6
2,2019,제주,10.2,null,41,46,8.4,7.4,12.5,null,null
3,2016,경남,3.7,1,56,61,3.9,3.1,5.9,4,0.8
4,2016,충남,3.6,2.1,34,42,3.3,2.9,4.8,8,1.9


In [56]:
# 병원 데이터 필요한 컬럼만 남기기 (연도, 시도, 종합병원, 일반병원)
hospital_df = hospital_df[['연도', '시도', '병의원_종합병원', '병의원_일반병원']]
hospital_df = hospital_df.rename(columns={
    '병의원_종합병원': '종합병원',
    '병의원_일반병원': '일반병원'
})

hospital_df.head()

,연도,시도,종합병원,일반병원
0,2015,서울 Seoul,56,217
1,2015,부산 Busan,28,130
2,2015,대구 Daegu,12,114
3,2015,인천 Incheon,19,55
4,2015,광주 Gwangju,22,75


#### 3. 데이터 분석 (EDA & 상관관계 분석)

#### 4. 시각화